# Import Libraries

In [ ]:
import json
import os
import xmltodict
import re
import time
import pandas as pd
from jsonpath import jsonpath
import sys
import warnings
from tqdm.notebook import tqdm

# Create the preprocess handler function

In [ ]:
data = {
}
cnt = 0
bigErr = 0
def processXmlFile(completePath):
  with open(completePath) as f:
    global cnt, data, bigErr
    cnt += 1

    xmlstring = f.read()
    dataDict = xmltodict.parse(xmlstring, xml_attribs = True)
    try:
      dataDict = dataDict["Document"]
    except:
      bigErr += 1
      return

    # Main topic of ques
    try:
      topic = dataDict["Focus"] + "|"+str(cnt)
    except Exception as e:
      warnings.warn("error in parsing Focus"+ str(e))
      topic = "UNKNOWN" + "|"+str(cnt)
    
    # Type of topic
    try:
      group = dataDict["FocusAnnotations"]["UMLS"]["SemanticGroup"]
    except Exception as e:
      warnings.warn("error in parsing group/type of topic"+ str(e))
      group = "UNKNOWN"
    
    # Synonyms
    try:
      synonyms = dataDict["FocusAnnotations"]["Synonyms"]["Synonym"]
    except Exception as e:
      warnings.warn("error in parsing QA"+ str(e))
      synonyms = []

    
    try:
      QAs_raw = dataDict["QAPairs"]["QAPair"]
      QAs = []
      for QA_i in QAs_raw:
        QA = {}
        QA["type"] = QA_i["Question"]["@qtype"]
        QA["question"] = QA_i["Question"]["#text"]
        QA["answer"] = QA_i["Answer"]
        QAs.append(QA)      

    except Exception as e:
      warnings.warn("error in parsing QA"+ str(e))
      QAs = []



    data[topic] = {
        "group":group,
        "synonyms":synonyms,
        "QAs":QAs,
    }

    
    '''
    for qaPair in listOfQA:
      try:
        # remove the extra spaces from answer with single space
        x = re.sub(' +', ' ', qaPair['Answer'])
        x = re.sub('Key Points', "", x)
        x = x.replace("\n", "").replace("-", "")
        data['Answers'].append( x )
        data['Questions'].append( qaPair['Question'] )
        data['Focus'].append(focus)
      except:
        return
        '''

# Preprocessing Loop

In [ ]:
foldersWithEmptyAnswers = [
  "10_MPlus_ADAM_QA", # These three removed due to copyright!
    "11_MPlusDrugs_QA",
    "12_MPlusHerbsSupplements_QA",
    "readme.txt", # As it does not contain any QAs
    "QA-TestSet-LiveQA-Med-Qrels-2479-Answers.zip", # will use it later,
    "ProcessedData.csv"
]

BASE_PATH = "/content/drive/MyDrive/Group_Project_Shang/Data/MedQuAD-master"
for folder in tqdm(os.listdir(BASE_PATH)):
  if folder in foldersWithEmptyAnswers:
    continue
  else:
    print("Processing folder:", folder)
    start = time.time()

    for xmlFileName in os.listdir( os.path.join(BASE_PATH, folder) ):
      completePath = os.path.join(BASE_PATH, folder, xmlFileName)
      # print("Processing File:", xmlFileName)
      processXmlFile(completePath)
      
      # print("Took", time.time() - start)
print("BIG ERRORS", bigErr)

# Saving 

In [ ]:
import json


with open('/content/drive/MyDrive/Group_Project_Shang/Data/medQuad_preprocessed/train.json', 'w') as f:
    json.dump(data, f)

# Visualizing

In [ ]:
data[list(data.keys())[0]].keys()

In [ ]:
data[list(data.keys())[0]]["QAs"][0].keys()